In [3]:
import pandas as pd

# Load the dataset
file_path = "../datasets/combined.parquet"
data = pd.read_parquet(file_path)

# Display the first few rows of the dataset for a quick overview
data.head()

,sentence,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1.0
1,dem rep. totally nails why congress is falling...,0.0
2,eat your veggies: 9 deliciously different recipes,0.0
3,inclement weather prevents liar from getting t...,1.0
4,mother comes pretty close to using word 'strea...,1.0


In [4]:
# Checking for any null values in the dataset
null_check = data.isnull().sum()

# Checking the distribution of the 'is_sarcastic' column
label_distribution = data["is_sarcastic"].value_counts(normalize=True)

null_check, label_distribution

(sentence        0
 is_sarcastic    0
 dtype: int64,
 is_sarcastic
 0.0    0.521391
 1.0    0.478609
 Name: proportion, dtype: float64)

In [5]:
import re
from sklearn.model_selection import train_test_split

# Data cleaning: removing special characters and escape sequences from the sentences
data["sentence"] = data["sentence"].apply(lambda x: re.sub(r"[\n\r\t]+", " ", x))

# Splitting the dataset into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Showing the size of each split
train_size, val_size, test_size = len(train_data), len(val_data), len(test_data)
train_size, val_size, test_size

(28322, 6069, 6070)

In [6]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


class SarcasticSentencesDataset(Dataset):
    """
    A custom PyTorch Dataset for the sarcastic sentences dataset.
    """

    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, item):
        sentence = str(self.sentences[item])
        label = self.labels[item]

        # Encoding the sentences using the tokenizer
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",  # Return PyTorch tensors
            truncation=True,
        )

        return {
            "sentence": sentence,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }


# Constants
MAX_LEN = 128  # Maximum length of the tokens list
BATCH_SIZE = 16

# Creating instances of the SarcasticSentencesDataset
train_dataset = SarcasticSentencesDataset(
    train_data["sentence"].to_numpy(),
    train_data["is_sarcastic"].to_numpy(),
    tokenizer,
    MAX_LEN,
)

val_dataset = SarcasticSentencesDataset(
    val_data["sentence"].to_numpy(),
    val_data["is_sarcastic"].to_numpy(),
    tokenizer,
    MAX_LEN,
)

test_dataset = SarcasticSentencesDataset(
    test_data["sentence"].to_numpy(),
    test_data["is_sarcastic"].to_numpy(),
    tokenizer,
    MAX_LEN,
)

# Creating the DataLoaders for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Checking the first batch from the train_loader
next(iter(train_loader))

/Users/remikalbe/.pyenv/versions/3.10.4/envs/iit_dl_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'sentence': ['democratic party gives bernie sanders bigger role in shaping its platform',
  'woman allegedly blows up pee sample in a 7-eleven microwave',
  'Short, yet pointless. This is the third post in a row where you have utterly and completely failed to address any of the content of what you have quoted from me. Lets see if we can make it FOUR!',
  'sixty seconds of art',
  "netflix's 'glow' trailer is an '80s wrestling-filled dream",
  'hoosier hostility: not the american way',
  'But what makes them any different from ant hills or beaver dams? skyscrapers are natural because humans are a part of nature and they built them.',
  'huffpost headline quiz: jan. 20 to jan. 26',
  'united airlines cracking down on emotional support spouses',
  'bush not heard from for over a month',
  "women aren't immune to sexism anywhere, even at the olympics",
  "man who plays devil's advocate really just wants to be asshole",
  "marti noxon poured her own life into 'to the bone,' a movie about a

In [14]:
import torch
from torch.utils.data import DataLoader
from transformers import (
    BertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup,
)
from torch.nn import CrossEntropyLoss
from tqdm import tqdm  # for displaying progress
import numpy as np

In [8]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [18]:
def train_epoch(model, data_loader, optimizer, device, scheduler, loss_fn, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for batch in tqdm(data_loader, total=len(data_loader)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids, attention_mask=attention_mask, labels=labels
        )

        loss = outputs.loss
        losses.append(loss.item())

        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.float() / n_examples, np.mean(losses)

In [19]:
def eval_model(model, data_loader, device, loss_fn, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, total=len(data_loader)):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids, attention_mask=attention_mask, labels=labels
            )

            loss = outputs.loss
            losses.append(loss.item())

            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    return correct_predictions.float() / n_examples, np.mean(losses)

In [20]:
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)  # Send the model to GPU if available

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3

# Total number of training steps
total_steps = len(train_loader) * num_epochs

# Scheduler for learning rate
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

# Loss function
loss_fn = CrossEntropyLoss()

# Training and Validation
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("-" * 10)

    train_acc, train_loss = train_epoch(
        model, train_loader, optimizer, device, scheduler, loss_fn, len(train_dataset)
    )

    print(f"Train loss {train_loss} accuracy {train_acc}")

    val_acc, val_loss = eval_model(model, val_loader, device, loss_fn, len(val_dataset))

    print(f"Validation loss {val_loss} accuracy {val_acc}")
    print()

# Save the model
torch.save(model.state_dict(), "sarcastic_model.pth")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/remikalbe/.pyenv/versions/3.10.4/envs/iit_dl_project/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
----------


100%|██████████| 1771/1771 [09:35<00:00,  3.08it/s]


Train loss 0.3479912940608385 accuracy 0.8380764126777649


100%|██████████| 380/380 [00:36<00:00, 10.51it/s]


...

Validation loss 0.3069017407950014 accuracy 0.8742791414260864

Epoch 3/3
----------


100%|██████████| 1771/1771 [09:50<00:00,  3.00it/s]


Train loss 0.08287450414135702 accuracy 0.9689993858337402


100%|██████████| 380/380 [00:34<00:00, 11.08it/s]


Validation loss 0.3856664270511828 accuracy 0.8782336711883545

